In [1]:

from keras.layers import Conv2D, Conv2DTranspose, UpSampling2D
from keras.layers import Activation, Dense, Dropout, Flatten, InputLayer
from keras.layers.normalization import BatchNormalization
from keras.callbacks import TensorBoard
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, rgb2gray
from skimage.io import imsave
import numpy as np
import os
import random
from skimage import img_as_ubyte


import tensorflow as tf

import PIL
from PIL import Image


Using TensorFlow backend.


In [ ]:
# resizeing images
#f = r'../256_ObjectCategories/input/'
#//for file in os.listdir(f):
#    f_img = f+"/"+file
#   img = Image.open(f_img)
#    img = img.resize((256,256))
#    img.save(f_img)

In [37]:


# Get images
#Images_new/Train/
images = []
for filename in os.listdir('../256_ObjectCategories/input/'):
    images.append(img_to_array(load_img('../256_ObjectCategories/input/' + filename)))
images = np.array(images, dtype=float)

# Set up train and test data
split = int(0.95*len(images))
images_train = images[:split]

# 1.0/255 -> 24 bit RGB color, [0-255] for each color channel -> 16.7m combinations -> human can recognize 2-10m 
images_train = 1.0/255*images_train

In [38]:
model = Sequential()
model.add(InputLayer(input_shape=(256, 256, 1)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))
model.add(UpSampling2D((2, 2)))
model.compile(optimizer='rmsprop', loss='mse', metrics=['accuracy'])

In [39]:
# Image datagenerator, adjust setting for img
datagen = ImageDataGenerator(
        shear_range=0.2,
        zoom_range=0.2,
        rotation_range=20,
        horizontal_flip=True)

# Generate training data
batch_size = 29 

def image_a_b_gen(batch_size):
    for batch in datagen.flow(images_train, batch_size=batch_size):
        lab_batch = rgb2lab(batch)
        # extract b/w layer for x_batch, two color layers for y_batch
        X_batch = lab_batch[:,:,:,0]
        Y_batch = lab_batch[:,:,:,1:] / 128
        yield (X_batch.reshape(X_batch.shape+(1,)), Y_batch)


trainedmodel = model.fit_generator(image_a_b_gen(batch_size), epochs=50, steps_per_epoch=33)



Epoch 1/50
33/33 [==============================] - 55s 2s/step - loss: 0.1791 - accuracy: 0.5544
Epoch 2/50
33/33 [==============================] - 52s 2s/step - loss: 0.0132 - accuracy: 0.6201
Epoch 3/50
33/33 [==============================] - 53s 2s/step - loss: 0.0150 - accuracy: 0.6091
Epoch 4/50
33/33 [==============================] - 52s 2s/step - loss: 0.0131 - accuracy: 0.6186
Epoch 5/50
33/33 [==============================] - 52s 2s/step - loss: 0.0142 - accuracy: 0.6248
Epoch 6/50
33/33 [==============================] - 52s 2s/step - loss: 0.0264 - accuracy: 0.6112
Epoch 7/50
33/33 [==============================] - 52s 2s/step - loss: 0.0140 - accuracy: 0.6253
Epoch 8/50
33/33 [==============================] - 52s 2s/step - loss: 0.0134 - accuracy: 0.6348
Epoch 9/50
33/33 [==============================] - 52s 2s/step - loss: 0.0140 - accuracy: 0.6200
Epoch 10/50
33/33 [==============================] - 52s 2s/step - loss: 0.0139 - accuracy: 0.6168
Epoch 11/50
33/33 [

In [40]:
# Save model
model_json = model.to_json()
with open("modelNew_resized2.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("modelNew_resized2.h5")

In [41]:

# Test images 
# Lab color, L -> grayscale layer, a b -> color layers (green-red, blue-yellow)
Xtest = rgb2lab(1.0/255*images[split:])[:,:,:,0]  # [:,:,:,0] -> select grayscale layer
Xtest = Xtest.reshape(Xtest.shape+(1,))
Ytest = rgb2lab(1.0/255*images[split:])[:,:,:,1:] # [:,:,:,1] -> select color layer

# ab spectrum [-128,128] .. divide by 128 to get [-1,1]
Ytest = Ytest / 128
print(model.evaluate(Xtest, Ytest, batch_size=batch_size))

96/96 [==============================] - 4s 41ms/step
[0.010830978009228906, 0.7158740162849426]


In [43]:
# colorizer section 
# init array and add img to color
images_to_color = []
for filename in os.listdir('Images_new/Test/'):
    images_to_color.append(img_to_array(load_img('Images_new/Test/'+filename)))

# covert color space, reshape img
images_to_color = np.array(images_to_color, dtype=float)
images_to_color = rgb2lab(1.0/255*images_to_color)[:,:,:,0]
images_to_color = images_to_color.reshape(images_to_color.shape+(1,))

# Test model with predict function
output = model.predict(images_to_color)

# covert color value back from [-1,1] to [-128,128] which is Lab color spectrum
output = output * 128

# Output colorizations
for i in range(len(output)):
    
    # make empty 256x256 with 3 layera
    cur = np.zeros((256, 256, 3))
    # copy grayscale layer to new obj
    cur[:,:,0] = images_to_color[i][:,:,0]
    # copy color layer to new obj
    cur[:,:,1:] = output[i]
    # save img in directory
    imsave("Images_new/Result/img_"+str(i)+".png", lab2rgb(cur))
    #imsave("Images_new/Result/img_"+str(i)+".png", img_as_ubyte(cur))

Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
